In [1]:
import sys
import torch as t
import torch
import numpy as np
from matplotlib import pyplot as plt
import json
from itertools import product
sys.path.append('/workspace/wilson/Finite-groups/src')
from model import MLP3, MLP4, InstancedModule
from utils import *
from group_data import *
from model_utils import *
from group_utils import *
from jaxtyping import Float
from typing import Union
from einops import repeat
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
import plotly.graph_objects as go
import copy
import math
from itertools import product
from llc import *
import pandas as pd
%load_ext autoreload
%autoreload 2

In [106]:
#path = '/workspace/models/2024-08-14_21-24-30_gapS_5_'
# path = '/workspace/models/2024-08-16_20-45-42_F11_MLP3_256'
#path = '/workspace/models/2024-08-16_22-14-00_F11_MLP3_256_wd2e-4'
path = '/workspace/models/2024-08-16_22-53-29_SL2_5_MLP3_256'
#path = '/workspace/models/2024-08-22_00-54-00_SL2_5_MLP3_512'

models, params = load_models(path, sel='final')

In [107]:
data = GroupData(params)

Intersection size: 14400/14400 (1.00)
Added 14400 elements from intersection
Added 0 elements from group 0: smallgrp(120, 5)
Taking random subset: 5760/14400 (0.40)
Train set size: 5760/14400 (0.40)


In [108]:
loss_dict = test_loss(models[-1].to(device), data)

In [109]:
loss_dict['G0_loss'].min().item(), loss_dict['G0_loss'].argmin().item()

(0.05674691125750542, 19)

In [110]:
instance = loss_dict['G0_loss'].argmin().item()

In [111]:
model = models[-1][instance].to(device)
lneurons, rneurons = model.get_neurons()
unembed = model.unembedding.data.detach()
lneurons, rneurons, unembed = lneurons.squeeze(0).to('cpu'), rneurons.squeeze(0).to('cpu'), unembed.squeeze(0).to('cpu')

In [112]:
irreps = data.groups[0].get_real_irreps()

In [113]:
irreps.keys()

dict_keys(['1d-0', '4d-0', '4d-1', '3d-0', '3d-1', '4d-2', '8d-0', '5d-0', '12d-0'])

In [114]:
irrep_bases = dict()
for name, irrep in irreps.items():
    irrep = einops.rearrange(irrep, 'N d1 d2 -> N (d1 d2)')
    U, S, V = torch.svd(irrep)
    nonzero = S > 1e-5
    irrep_bases[name] = U[:,nonzero]

In [115]:
for k, v in irrep_bases.items():
    print(k, v.shape)

1d-0 torch.Size([120, 1])
4d-0 torch.Size([120, 4])
4d-1 torch.Size([120, 4])
3d-0 torch.Size([120, 9])
3d-1 torch.Size([120, 9])
4d-2 torch.Size([120, 16])
8d-0 torch.Size([120, 16])
5d-0 torch.Size([120, 25])
12d-0 torch.Size([120, 36])


In [116]:
lexpl = dict()
for name, basis in irrep_bases.items():
    proj = basis.T @ lneurons
    lexpl[name] = proj.norm(dim=0)**2 / (lneurons.norm(dim=0)**2 + 1e-8)
    #lexpl[name] = lexpl[name].nan_to_num(-1.)

In [117]:
lexpl_total, lirrep = t.stack(list(lexpl.values()), dim=0).max(dim=0)

In [118]:
lexpl_total

tensor([9.6526e-01, 9.6479e-01, 9.9998e-01, 2.1284e-12, 9.9998e-01, 4.3271e-07,
        1.3073e-03, 9.9999e-01, 2.3386e-10, 0.0000e+00, 9.9993e-01, 1.0090e-07,
        9.9999e-01, 1.8392e-05, 9.9998e-01, 9.9998e-01, 9.6624e-01, 9.6942e-01,
        9.6445e-01, 9.9999e-01, 2.9427e-36, 9.9998e-01, 9.7221e-01, 1.5505e-01,
        2.0718e-08, 9.9999e-01, 9.6206e-01, 0.0000e+00, 1.4253e-02, 0.0000e+00,
        9.6459e-01, 5.8649e-27, 9.6874e-01, 9.6685e-01, 7.1095e-03, 9.6357e-01,
        9.4334e-01, 9.7287e-01, 9.6502e-01, 9.6088e-01, 9.7725e-01, 9.6819e-01,
        7.0039e-18, 9.6431e-01, 9.9999e-01, 9.3428e-07, 0.0000e+00, 9.6488e-01,
        9.7382e-15, 9.6426e-01, 9.9998e-01, 9.7209e-01, 9.9998e-01, 9.9996e-01,
        1.1178e-32, 9.9999e-01, 8.7067e-32, 3.6153e-06, 9.6504e-01, 8.9453e-27,
        9.7123e-01, 2.6629e-02, 1.8293e-03, 0.0000e+00, 5.1622e-01, 9.9999e-01,
        9.6935e-01, 9.6175e-01, 9.9998e-01, 1.3225e-02, 9.6642e-01, 9.6330e-01,
        9.9999e-01, 9.7611e-01, 9.6535e-

In [119]:
irrep_counts = defaultdict(lambda: 0)
for i in range(lexpl_total.shape[0]):
    if t.isnan(lexpl_total[i]):
        irrep_counts['dead'] += 1
    elif lexpl_total[i] < 0.95:
        irrep_counts['none'] += 1
    else:
        irrep_counts[list(lexpl.keys())[lirrep[i]]] += 1


In [131]:
','.join(map(str, (lexpl['4d-2'] > 0.9999).nonzero().flatten().tolist()))

'2,4,7,10,12,14,15,19,21,25,44,50,52,53,55,65,68,72,79,92,96,118,119,128,132,136,139,142,143,144,148,149,153,178,197,200,205,207,209,211,213,214,215,217,222,226,228,235,244,252,254'

In [125]:
lexpl['4d-2'][7]

tensor(1.0000)

In [126]:
irrep_counts

defaultdict(<function __main__.<lambda>()>,
            {'4d-1': 129, '4d-2': 52, 'none': 75})

In [97]:
irrep1_neurons = [i for i in range(lneurons.shape[-1]) if lexpl['4d-1'][i].item() > 0.95]

In [98]:
irrep2_neurons = [i for i in range(lneurons.shape[-1]) if lexpl['4d-2'][i].item() > 0.95]

In [99]:
none_neurons = [i for i in range(lneurons.shape[-1]) if  lexpl['4d-1'][i].item() < 0.90 and  lexpl['4d-2'][i].item() < 0.90]

In [100]:
lneurons[:,irrep1_neurons].norm(dim=0).mean()  # 4d-1

tensor(7.8797)

In [101]:
lneurons[:,irrep2_neurons].norm(dim=0).mean()  # 4d-2

tensor(19.2476)

In [102]:
lneurons[:,none_neurons].norm(dim=0).mean()  # none

tensor(0.0084)

In [25]:
len(none_neurons)

287

In [26]:
ablate_model = copy.deepcopy(model)
ablate_model.unembedding.data[0,none_neurons,:] = 0.
loss_dict = test_loss(model.to(device), data)
ablate_loss_dict = test_loss(ablate_model.to(device), data)
loss_dict['G0_loss'], ablate_loss_dict['G0_loss']

(tensor([0.0547], device='cuda:0'), tensor([0.0547], device='cuda:0'))

In [44]:
lexpl['10d-0'][lexpl['10d-0'] > 0.9].min()

tensor(0.9998)

In [27]:
none_neurons[:10]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [32]:
irrep1_neurons[:10]

[10, 12, 20, 27, 29, 32, 33, 36, 41, 50]

In [33]:
lneurons[:,10]

tensor([ 0.4543, -0.3215, -0.3866,  0.0658,  0.2849,  0.7146, -0.7001, -0.3519,
        -0.0293,  0.4451,  0.3903, -0.2266, -0.3949,  0.0404,  0.0277,  0.4426,
        -0.6026,  0.3797,  0.1790, -0.2919,  0.2380, -0.6539,  0.3580,  0.8155,
        -0.7410, -0.1023,  0.5109, -0.3375, -0.7893,  0.7420, -0.1855,  0.1518,
        -0.0982, -0.1477,  0.2011, -0.7619,  0.6268,  0.4548, -0.0048, -0.3954,
        -0.4225,  0.3286,  0.2775, -0.1781,  0.0290,  0.5364,  0.0485,  0.7303,
        -0.8946, -0.3909,  0.1954, -0.7090, -0.6683,  0.7218,  0.4182, -0.2136,
         0.5629,  0.4081,  0.0235, -0.6883, -0.3297,  0.3457,  0.0522,  0.0737,
         0.0315,  0.6407, -0.5912,  0.1984, -0.5789,  0.2211, -0.6088,  0.4773,
        -0.0368, -0.3619,  0.7157, -0.9290, -0.1772, -0.3477,  0.6074,  0.6088,
        -0.5665,  0.3696, -0.3070,  0.1151,  0.5380, -0.0693,  0.0309, -0.3814,
         0.2836,  0.1523,  0.3601,  0.2906, -0.1990, -0.0784, -0.4478, -0.3114,
         0.5049,  0.5742, -0.6757, -0.04